# Urban Mobility

[![Binder](http://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/ogd-at-lab/main?urlpath=lab/tree/notebooks/mobility.ipynb)

In [ ]:
import hvplot.pandas
import movingpandas as mpd
from datetime import timedelta
from utils.dataaccess import get_uber_movement_gdf, get_osm_traces

## Uber Movement

Source: https://movement.uber.com/explore/vienna/travel-times

© 2021 Copyright Uber Technologies. Data made available under the [Creative Commons, Attribution Non-Commercial](https://creativecommons.org/licenses/by-nc/3.0/us/) license

In [ ]:
gdf = get_uber_movement_gdf()

In [ ]:
def plot_uber(source_id, attribute_col, month, *args, **kwargs):
    return ( gdf[(gdf.sourceid==source_id) & (gdf.month==month)].dropna(subset=[attribute_col]).hvplot(
            geo=True, tiles='OSM', c=attribute_col, title=f'{attribute_col.title()} - Month: 2020-{month}', *args, **kwargs) * 
        gdf[gdf.index==source_id].hvplot(geo=True).opts(active_tools=['wheel_zoom']))

In [ ]:
COL = 'mean_travel_time'
ID = 1
plot_uber(ID, COL, 1) + plot_uber(ID, COL, 2) + plot_uber(ID, COL, 3)

In [ ]:
jan = gdf[gdf.month==1].groupby('sourceid').count()
feb = gdf[gdf.month==2].groupby('sourceid').count()
mar = gdf[gdf.month==3].groupby('sourceid').count() 

In [ ]:
joined = feb.join(mar, lsuffix='_feb', rsuffix='_mar')
diff = joined[f'{COL}_mar'] - joined[f'{COL}_feb'] 
diff.hvplot.hist(title='Histogram of destination area counts (March 2020 - Feb 2020)', xlim=(-300,300))

In [ ]:
joined = jan.join(feb, lsuffix='_jan', rsuffix='_feb')
diff = joined[f'{COL}_feb'] - joined[f'{COL}_jan'] 
diff.hvplot.hist(title='Histogram of destination area counts (Feb 2020 - Jan 2020)', xlim=(-300,300))

## OpenStreetMap Traces

Source: https://www.openstreetmap.org/traces

In [ ]:
gdf = get_osm_traces()
osm_traces = mpd.TrajectoryCollection(gdf, 'track_fid')
print(f'The OSM traces download contains {len(osm_traces)} tracks')

In [ ]:
for track in osm_traces: print(f'Track {track.id}: length={track.get_length():.0f}m')

In [ ]:
osm_traces = mpd.MinTimeDeltaGeneralizer(osm_traces).generalize(tolerance=timedelta(minutes=1))
osm_traces.hvplot(title='OSM Traces', line_width=7, width=700, height=500)

In [ ]:
osm_traces.get_trajectory(0).hvplot(title='Speed (m/s) along track', c='speed', cmap='RdYlBu',
                                    line_width=7, width=700, height=500, tiles='CartoLight', colorbar=True)

## Interactive Trajectory Generalization Application

This demo uses pn.interact() which may not work in Jupyter Lab. Use Jupyter Notebook instead if the application does not update when you change the input parameters. 

In [ ]:
import panel as pn
from pyproj import CRS

In [ ]:
def my_plot(traj, tolerance=0.001, generalizer='DouglasPeuckerGeneralizer'):
    if generalizer=='DouglasPeuckerGeneralizer':
        generalized = mpd.DouglasPeuckerGeneralizer(traj).generalize(tolerance=tolerance)
    else:
        generalized = mpd.MinDistanceGeneralizer(traj).generalize(tolerance=tolerance)
    generalized.add_speed(overwrite=True)
    return ( generalized.hvplot(title='Speed along trajectory', c='speed', cmap='Viridis', colorbar=True, clim=(0,20), line_width=10, width=500, height=400) + 
             generalized.df['speed'].hvplot.hist(title='Speed histogram', width=300, height=450) 
           )

In [ ]:
traj = osm_traces.get_trajectory(0).to_crs(CRS(31256))
kw = dict(traj=traj, tolerance=(0, 1000), generalizer=['DouglasPeuckerGeneralizer', 'MinDistanceGeneralizer'])
pn.interact(my_plot, **kw)